In [1]:

from database.sheetsorm.orm import SheetsORM
import os
SPREADSHEET_NAME  = os.getenv('SPREADSHEET_NAME','Pegasus - DEV')
USER_EMAIL = os.getenv('USER_EMAIL', 'pedrohenriquebraga735@gmail.com')

db = SheetsORM(scope=['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive'],
            credentials_file='database/credentials.json', spreadsheet_name=SPREADSHEET_NAME)
db.connect(create_if_not_exist=True)
try:
    db.share(USER_EMAIL,perm_type='user', role='owner')
except:
    pass





In [2]:

from datetime import datetime
from database.sheetsorm.decorators import entity, column

@entity()
class TBL_Account:
    @column(primary_key=True)
    def ID_Account(self) -> int:
       pass
    
    @column()
    def ID_AccountGroup(self) -> int:
       pass

    @column()
    def DT_CreatedDate(self) -> datetime:
         pass

    @column()
    def NR_InitialAmount(self) -> float:
         pass

    @column(length=250)
    def DS_Name(self) -> str:
        pass

    @column()
    def NR_Number(self) -> str:
        pass

    @column(length=250)
    def DS_BankOfficeNumber(self) -> str:
        pass

    @column(length=250)
    def DS_Bank(self) -> str:
        pass




In [3]:
repo = db.get_repository(TBL_Account)

lista  = repo.get_all()


In [4]:
a = repo.find(lambda x: x.DS_Name == 'Cartão de crédito')[0]
a.DS_Name = 'Cartão de DDD'
repo.update(a)
# setattr(a,'__str__', lambda self: f"<{self.__class__.__name__}( {', '.join([x for x in self._asdict().values()])} )>")

In [5]:
# repo.count(lambda x: x.DS_Name == 'opaaa')
repo.commit()


TypeError: Object of type datetime is not JSON serializable

In [23]:
from pandas import DataFrame
import pandas as pd
DTYPES = {
    'int': 'int64',
    'datetime': 'datetime64[ns]',
    'float': 'float',
    'bool': 'bool',
    'str': 'str'
}
df  = repo._df_copy.tail()
df.select_dtypes(include=['datetime64[ns]']).apply(lambda x: x.astype('category'))

temp_df  = DataFrame({
        column['name']: pd.Series(dtype=DTYPES[column['dtype']]) for column  in repo.model['__data']
    })

temp_df = temp_df.append(pd.DataFrame(repo.worksheet.get_all_records()))
for column in temp_df.columns:
    if temp_df[column].dtype == 'datetime64[ns]':
        temp_df[column] = pd.to_datetime(temp_df[column], format='%Y-%m-%d %H:%M:%S')
    else:
        temp_df[column] = temp_df[column].astype(temp_df[column].dtype)
temp_df.select_dtypes(include=['datetime64','datetime64[ns]','<M8[ns]']).columns.tolist()

# df = df.fillna('null')
# df['DT_CreatedDate'] = df['DT_CreatedDate'].astype(str)
# print(df['DT_CreatedDate'].dtype)
# df.values.tolist()
# repo.worksheet.update()

[]